In [1]:
#!/usr/bin/env python3
"""
iwasawa_invariants.py

Compute Iwasawa invariants (mu, lambda, nu) for a Z_p-extension from class-group growth data.

Model:
    v_p(h_n) = mu * p**n + lambda * n + nu       (for large n)

Author: ChatGPT (GPT-5 Thinking mini)
Date: 2025-10-31
"""

from typing import List, Optional, Tuple, Dict
from fractions import Fraction
import math
import sys

# --------------------------
# Basic utilities
# --------------------------

def v_p_of_int(x: int, p: int) -> int:
    """Return the p-adic valuation v_p(x). By convention v_p(0)=+inf -> we return a large sentinel."""
    if x == 0:
        # sentinel: very large
        return 10**9
    if p <= 1:
        raise ValueError("p must be a prime > 1")
    v = 0
    xx = abs(x)
    while xx % p == 0:
        xx //= p
        v += 1
    return v

def ensure_list_of_ints(seq):
    return [int(x) for x in seq]

# --------------------------
# Core algebra: recover mu,lambda,nu
# --------------------------

def compute_iwasawa_invariants_from_class_numbers(p: int, class_numbers: List[int],
                                                  assume_vp_provided: bool = False,
                                                  start_layer_for_fit: Optional[int] = None
                                                  ) -> Dict:
    """
    Given prime p and class numbers h_n for layers n=0..N (list length N+1),
    return an attempt at (mu, lambda, nu) together with diagnostics.

    If assume_vp_provided is True, then class_numbers is interpreted
    as the list of v_p(h_n) already.
    """
    if p <= 1:
        raise ValueError("p must be a prime > 1")
    if len(class_numbers) < 3:
        raise ValueError("Need at least 3 layers (n=0..2) to attempt a fit.")

    # compute valuations
    if assume_vp_provided:
        vlist = ensure_list_of_ints(class_numbers)
    else:
        vlist = [v_p_of_int(h, p) for h in class_numbers]

    N = len(vlist) - 1  # highest layer index
    # default start layer: use earliest possible but allow user override to skip small n
    if start_layer_for_fit is None:
        start = 0
    else:
        start = int(start_layer_for_fit)
        if start < 0 or start > N-1:
            raise ValueError("start_layer_for_fit out of range")

    # Compute D_n = v_{n+1} - p * v_n for n = start .. N-1
    D = []
    ns = []
    for n in range(start, N):
        Dn = vlist[n+1] - p * vlist[n]
        D.append(Dn)
        ns.append(n)

    # We expect D_n to fit an affine linear model:
    # D_n = A * n + B,
    # where A = -lambda*(p-1) and B = lambda + (1-p)*nu
    # Solve for A,B (rational) by ordinary least squares on small data, but we prefer exact rational
    # If there are >=2 points and they are consistent, derive exact rational A,B using two distinct points.

    def fit_line_exact(xs, ys) -> Tuple[Fraction, Fraction]:
        # Given at least two points, attempt to compute exact slope/intercept as fractions
        if len(xs) == 0:
            raise ValueError("No points to fit")
        if len(xs) == 1:
            # slope 0 and intercept = y
            return Fraction(0), Fraction(ys[0])
        # Try exact solution using first two points with distinct x
        for i in range(len(xs)):
            for j in range(i+1, len(xs)):
                if xs[j] != xs[i]:
                    # slope m = (y2-y1)/(x2-x1)
                    m = Fraction(ys[j] - ys[i], xs[j] - xs[i])
                    b = Fraction(ys[i]) - m * xs[i]
                    # verify all points agree
                    ok = True
                    for k in range(len(xs)):
                        if Fraction(ys[k]) != m * xs[k] + b:
                            ok = False
                            break
                    if ok:
                        return m, b
                    # if not consistent exact, fallback to rational least-squares fit
                    # break out to do least-squares
        # Fallback: rational least-squares via normal equations (may not be exact integer)
        Sx = sum(Fraction(x) for x in xs)
        Sy = sum(Fraction(y) for y in ys)
        Sxx = sum(Fraction(x) * x for x in xs)
        Sxy = sum(Fraction(x) * y for x, y in zip(xs, ys))
        n = Fraction(len(xs))
        denom = (n * Sxx - Sx * Sx)
        if denom == 0:
            # vertical or degenerate, return zero slope
            m = Fraction(0)
        else:
            m = (n * Sxy - Sx * Sy) / denom
        b = (Sy - m * Sx) / n
        return m, b

    mA, bB = fit_line_exact(ns, D)  # A = mA, B = bB (as Fractions)

    # Now recover lambda and nu:
    # A = -lambda*(p-1)  => lambda = -A/(p-1)
    # B = lambda + (1-p)*nu  => nu = (lambda - B)/(p-1)
    p_minus_1 = p - 1
    lambda_frac = -mA / p_minus_1
    # require lambda integer if model exact
    lambda_is_integer = (lambda_frac.denominator == 1)
    lambda_int = int(lambda_frac) if lambda_is_integer else None

    # compute nu
    nu_frac = (lambda_frac - bB) / p_minus_1
    nu_is_integer = (nu_frac.denominator == 1)
    nu_int = int(nu_frac) if nu_is_integer else None

    # compute mu from highest n (to reduce pre-asymptotic noise)
    # mu = (v_n - lambda*n - nu) / p^n
    mu_candidates = []
    for n in range(N, max(N-3, -1), -1):  # try last up to 4 layers backwards
        numerator = Fraction(vlist[n]) - lambda_frac * n - nu_frac
        denom = Fraction(p ** n)
        mu_frac = numerator / denom
        if mu_frac.denominator == 1:
            mu_candidates.append(int(mu_frac))
        else:
            # also allow mu 0 if near zero
            mu_candidates.append(float(mu_frac))  # may be fractional => indicates non-exact fit

    # choose best mu candidate if integer found
    mu_int = None
    for c in mu_candidates:
        if isinstance(c, int):
            mu_int = c
            break

    # compute residuals using found rationals
    residuals = []
    for n in range(len(vlist)):
        predicted = lambda_frac * n + nu_frac + Fraction(0)  # will add mu*p^n if mu known
        if mu_int is not None:
            predicted += Fraction(mu_int) * (p ** n)
        else:
            # use rational approximation using mu_frac from last attempted n (not ideal)
            predicted += mu_candidates[0] * (p ** n) if len(mu_candidates) > 0 else Fraction(0)
        residuals.append(Fraction(vlist[n]) - predicted)

    # compute simple diagnostics
    max_abs_residual = max(abs(float(r)) for r in residuals)
    consistent_linear_D = all(Fraction(D[i]) == mA * ns[i] + bB for i in range(len(D)))

    result = {
        "p": p,
        "layers_provided": len(class_numbers),
        "v_p_list": vlist,
        "fit_start_layer": start,
        "D_values": list(zip(ns, D)),
        "A": mA,
        "B": bB,
        "lambda_frac": lambda_frac,
        "lambda_is_integer": lambda_is_integer,
        "lambda_int": lambda_int,
        "nu_frac": nu_frac,
        "nu_is_integer": nu_is_integer,
        "nu_int": nu_int,
        "mu_candidates": mu_candidates,
        "mu_int": mu_int,
        "residuals": residuals,
        "max_abs_residual": max_abs_residual,
        "consistent_D_exact": consistent_linear_D,
    }
    return result

# --------------------------
# Example / synthetic data generator
# --------------------------

def synthetic_class_numbers_from_invariants(p: int, mu:int, lam:int, nu:int, layers:int) -> List[int]:
    """
    Generate synthetic v_p(h_n) data from invariants and then produce sample class numbers h_n
    with p-adic valuation equal to v_n (we set h_n = p**v_n to keep canonical).
    """
    vlist = []
    for n in range(layers):
        v = mu * (p ** n) + lam * n + nu
        vlist.append(int(v))
    # produce class numbers as p**v (simple canonical choice)
    class_numbers = [p ** v for v in vlist]
    return class_numbers

# --------------------------
# Main CLI-ish behavior
# --------------------------

EXPOSITION = r"""
# Exposition: Iwasawa Theory (structured, layered)

**Warning:** The exposition below is a compact but rigorous survey intended for mathematicians comfortable
with algebraic number theory. It covers background, definitions, structural theorems, and the
interpretation of Iwasawa invariants λ, μ, ν in the classical cyclotomic Z_p-extension context,
followed by notes about generalizations, computational methodology, and caveats.

(1) **Foundational setup.**
 Let K be a number field. Fix a rational prime p. A **Z_p-extension** of K is an infinite Galois extension
K_\infty/K whose Galois group Γ = Gal(K_\infty/K) is isomorphic (topologically) to the additive
p-adic group Z_p. Equivalently Γ ≅ lim← Z/p^nZ. A standard example: for K = Q, the cyclotomic Z_p-extension Q_\infty
is obtained by adjoining all p^n-th power roots of unity (when p is odd) and taking the unique Z_p-extension.

 Notation: Let K_n denote the intermediate subfield of K_\infty with [K_n:K] = p^n. Denote by Cl(K_n) the ideal class group.
 We are primarily interested in the p-part of Cl(K_n), e.g. A_n := Cl(K_n) ⊗ Z_p (equivalently the Sylow p-subgroup).

(2) **Iwasawa module and structure theorem.**
 The inverse limit A := lim← A_n under norm maps becomes a module over the Iwasawa algebra Λ := Z_p[[Γ]].
 Choosing a topological generator γ of Γ identifies Λ ≅ Z_p[[T]] via γ ↦ 1+T. The celebrated **structure theorem**
for finitely generated torsion Λ-modules M (analogue of the structure theorem for finitely generated modules
over PID) states:
  M ≅ (⊕_i Λ/(p^{μ_i})) ⊕ (⊕_j Λ/(f_j(T)^{e_j})) ⊕ (Λ^r)
 where f_j(T) are distinguished (Weierstrass) polynomials (monic with coefficients divisible by p).
 For the basic cyclotomic setup where A is torsion over Λ, the free rank r = 0 and the p-power primary part contributes to μ.

(3) **Iwasawa invariants λ, μ, ν (classical formulation).**
 If A is a torsion Λ-module, one defines the invariants (μ, λ, ν) from the characteristic ideal:
  char_Λ(A) = p^{μ} · f(T)
 where f(T) is a distinguished polynomial of degree λ (counted with multiplicity), and ν is an integer
capturing p-adic unit factor normalization / finite index terms. Analytically, the growth of p-part of class numbers h_p(K_n)
is then governed for sufficiently large n by the formula
  v_p(h(K_n)) = μ p^n + λ n + ν   (Iwasawa's classical asymptotic)
where v_p denotes p-adic valuation. This formula is a precise manifestation of the algebraic structure of A.

 Intuitively:
  - μ measures "exponential" growth in p^n (coming from p-power torsion in Λ-module).
  - λ measures "linear" growth in n (coming from the degree of the distinguished polynomial factor).
  - ν is a constant shift for the particular tower.

(4) **Interpretation and significance.**
  - If μ = 0, the p-part of class numbers grows at most linearly with the layer index n.
  - Iwasawa conjectured (and in many cyclotomic cases proved by Ferrero–Washington for abelian extensions of Q)
    that μ = 0 for cyclotomic Z_p-extensions of abelian number fields (p odd). For general number fields μ can be > 0.
  - λ often relates to the number of Z_p-extensions, the zeroes of p-adic L-functions, and subtle arithmetic invariants.

(5) **Computational strategy (practical).**
  - Direct computation of class groups in high layers quickly becomes infeasible; practical computations are
    limited to small n. However, given class numbers h(K_n) or their p-adic valuations v_p(h(K_n)), one can attempt
    an algebraic extraction of the invariants by using the growth formula.
  - Algebraic rearrangement: Let y_n := v_p(h(K_n)). Define D_n := y_{n+1} - p·y_n. Using the model:
        D_n = -λ(p-1) n + (λ + (1-p)ν) .
    Hence D_n is affine-linear in n with slope A = -λ(p-1) and intercept B = λ + (1-p)ν. This transformation removes
    the μ·p^n leading term and reduces the extraction to a (rational) linear fit for D_n vs n. After recovering λ and ν
    (as rational numbers that should be integers if the asymptotic regime is reached), one recovers μ by
    μ = (y_n - λ n - ν) / p^n for a large enough n (integer if consistent).
  - Important caveat: the formula holds only for sufficiently large n (past the "stability" threshold). For small n
    additional sporadic factors, local units, ramification effects can make the raw data deviate.

(6) **Algorithmic robustness and diagnostics.**
  - Use valuation data y_n rather than raw class numbers to avoid overflow and to focus on p-primary behaviour.
  - Fit D_n = A n + B exactly if possible (rational arithmetic). Check integrality constraints:
      lambda = -A/(p-1) should be integer,
      nu = (lambda - B)/(p-1) should be integer,
      mu computed from the largest available n should be an integer.
  - If integrality fails, either (i) the asymptotic regime is not reached; (ii) input data inconsistent; or (iii) the model assumptions (torsion / finiteness) don't hold.

(7) **Extensions and theoretical subtleties.**
  - For non-cyclotomic Z_p-extensions, or when A has non-torsion submodules, the simple formula might not hold.
  - The "characteristic power series" may factor with multiplicities and nontrivial Galois-module structure; descriptions
    of λ, μ connect to zeroes of p-adic L-functions (Iwasawa Main Conjecture / theorem in many cases).
  - For computational work one should always annotate layers with ramification data, unit ranks, and whether K_n contain p-th roots of unity.

(8) **Examples and edge cases.**
  - Example: for many abelian fields over Q in cyclotomic towers, μ = 0 (Ferrero-Washington). Then
        y_n = λ n + ν
    eventually linear, which is easy to detect by looking at first differences y_{n+1}-y_n.
  - When data is noisy (small n), the algorithm may give fractional λ or ν; interpret these as "no stable fit".

(9) **Practical advice.**
  - Produce valuations y_n for as many layers as computational resources allow.
  - Inspect D_n visually: if D_n is linear for several successive n, you likely are in the stable regime.
  - Cross-check by computing mu from different large n and checking consistency.

(10) **Conclusion.**
  The arithmetic content encoded by (μ,λ,ν) summarizes deep arithmetic of the p-adic tower. Algorithmically
  the above algebraic manipulations strip away the exponential term and reduce a nonlinear-fitting problem to
  a rational linear algebra problem that is stable and exact when the data sits in the asymptotic regime.
"""

def save_exposition(filename="iwasawa_exposition.md"):
    with open(filename, "w", encoding="utf-8") as f:
        f.write(EXPOSITION)
    return filename

def demo():
    # Synthetic example where mu=1, lambda=2, nu=3, p=3, layers=6
    p = 3
    mu_true = 1
    lam_true = 2
    nu_true = 3
    layers = 7
    print("Generating synthetic class numbers with invariants mu={}, lambda={}, nu={} (p={})".format(mu_true, lam_true, nu_true, p))
    cls = synthetic_class_numbers_from_invariants(p, mu_true, lam_true, nu_true, layers)
    print("Synthetic class numbers (canonical p^v form):", cls)
    result = compute_iwasawa_invariants_from_class_numbers(p, cls, assume_vp_provided=False, start_layer_for_fit=0)
    print("\nFit result summary:")
    print("p:", result["p"])
    print("layers provided:", result["layers_provided"])
    print("v_p list:", result["v_p_list"])
    print("A (slope for D_n):", result["A"])
    print("B (intercept for D_n):", result["B"])
    print("lambda (fraction):", result["lambda_frac"], "is_integer:", result["lambda_is_integer"])
    print("lambda_int:", result["lambda_int"])
    print("nu (fraction):", result["nu_frac"], "is_integer:", result["nu_is_integer"])
    print("nu_int:", result["nu_int"])
    print("mu candidates (from last layers):", result["mu_candidates"])
    print("chosen mu_int:", result["mu_int"])
    print("max absolute residual (in valuations):", result["max_abs_residual"])
    print("consistent D exact:", result["consistent_D_exact"])
    print("\nExposition saved to file:", save_exposition())

if __name__ == "__main__":
    # If run as script, execute demonstration
    demo()


Generating synthetic class numbers with invariants mu=1, lambda=2, nu=3 (p=3)
Synthetic class numbers (canonical p^v form): [81, 6561, 43046721, 150094635296999121, 78551672112789411833022577315290546060373041, 139008452377144732764939786789661303114218850808529137991604824430036072629766435941001769154109609521811665540548899435521, 9510722527101357340525747647537197965886751007681553994034610554442328346281493944874993791266071800329593708229097712659875810002117855554604673436499654209708323639592014707001230914047790029222853540449516608542614875072052519793441054033984528508073352559714628517355485426276535664871435922971220253912946391119645605633303729459373410643352911406881]

Fit result summary:
p: 3
layers provided: 7
v_p list: [4, 8, 16, 36, 92, 256, 744]
A (slope for D_n): -4
B (intercept for D_n): -4
lambda (fraction): 2 is_integer: True
lambda_int: 2
nu (fraction): 3 is_integer: True
nu_int: 3
mu candidates (from last layers): [1, 1, 1]
chosen mu_int: 1
max absolute resid